# Char2id

In [1]:
def file_padding(i):
    if i < 10: return '00000' + str(i)
    elif i < 100: return '0000' + str(i)
    elif i < 1000: return '000' + str(i)
    elif i < 10000: return '00' + str(i)
    elif i < 100000: return '0' + str(i)
    else: return str(i)

In [2]:
def get_path(path, fname, filenum, format_):
    return path+fname+filenum+format_

## Create kai_labels.csv

In [8]:
from tqdm import trange

path = "E:/한국어 음성데이터/KaiSpeech/"
fname = 'KaiSpeech_'
TOTAL_NUM = 622545
label_list = []
label_freq = []
print('started...')
for filenum in trange(1,TOTAL_NUM):
    f = open(get_path(path,fname,file_padding(filenum),".txt"))
    sentence = f.readline()
    f.close()
    for ch in sentence:
        if ch not in label_list:
            label_list.append(ch)
            label_freq.append(1)
        else:
            label_freq[label_list.index(ch)] += 1
label_freq, label_list = zip(*sorted(zip(label_freq, label_list), reverse=True))
label = {'id':[], 'char':[], 'freq' :[]}
for idx, ch_freq in enumerate(zip(label_list, label_freq)):
    ch, freq = ch_freq
    label['id'].append(idx)
    label['char'].append(ch)
    label['freq'].append(freq)

started...


100%|████████████████████████████████████████████████████████████████████████| 622544/622544 [03:02<00:00, 3409.03it/s]


NameError: name 'pd' is not defined

## Create KaiSpeech_label_FileNum

In [3]:
import pandas as pd
char2id = dict() # [ch] = id
id2char = dict() # [id] = ch

df_labels = pd.read_csv('kai_labels.csv', encoding='cp949')
id_list = df_labels['id']
ch_list = df_labels['char']

for idx, ch in enumerate(ch_list):
    char2id[ch] = id_list[idx]
    id2char[id_list[idx]] = ch

In [5]:
from tqdm import trange

path = "E:/한국어 음성데이터/KaiSpeech/"
fname = 'KaiSpeech_'
label_fname = 'KaiSpeech_label_'
TOTAL_NUM = 622545
START_POINT = 1
print('started...')
for filenum in trange(START_POINT,TOTAL_NUM):
    f = open(get_path(path,fname,file_padding(filenum),".txt"))
    sentence = f.readline()
    labels = str()
    f.close()
    for idx, ch in enumerate(sentence):
        if idx == 0: labels += str(char2id[ch])
        else : labels += (' ' + str(char2id[ch]))
    f = open(get_path(path,label_fname,file_padding(filenum),".txt"),"w")
    f.write(labels)
    f.close()
print("Ended!!!")

started...


100%|████████████████████████████████████████████████████████████████████████| 105321/105321 [7:35:17<00:00,  1.79it/s]


Ended!!!


In [3]:
import pandas as pd
TOTAL_NUM = 622545
TRAIN_NUM = int( 622545 * 0.8 )
TEST_NUM = TOTAL_NUM - TRAIN_NUM
train_data_list = {'audio':[], 'label':[]}
test_data_list = {'audio':[], 'label':[]}
kai_labels = pd.read_csv("kai_labels.csv", encoding='cp949')
rare_labels = kai_labels['char'][2037:]

In [4]:
from tqdm import trange

fname = 'KaiSpeech_'
label_fname = 'KaiSpeech_label_'

audio_paths = []
label_paths = []

for filenum in trange(1, TOTAL_NUM):
    audio_paths.append(fname+file_padding(filenum)+".pcm")
    label_paths.append(label_fname+file_padding(filenum)+".txt")

100%|██████████████████████████████████████████████████████████████████████| 622544/622544 [00:01<00:00, 318302.03it/s]


In [5]:
def head(list_):
    print(list_[:5])
head(audio_paths)
head(label_paths)

['KaiSpeech_000001.pcm', 'KaiSpeech_000002.pcm', 'KaiSpeech_000003.pcm', 'KaiSpeech_000004.pcm', 'KaiSpeech_000005.pcm']
['KaiSpeech_label_000001.txt', 'KaiSpeech_label_000002.txt', 'KaiSpeech_label_000003.txt', 'KaiSpeech_label_000004.txt', 'KaiSpeech_label_000005.txt']


In [6]:
import random
data_paths = list(zip(audio_paths, label_paths))
random.shuffle(data_paths)
audio_paths, label_paths = zip(*data_paths)

In [7]:
head(audio_paths)
head(label_paths)

('KaiSpeech_078903.pcm', 'KaiSpeech_449461.pcm', 'KaiSpeech_620819.pcm', 'KaiSpeech_103058.pcm', 'KaiSpeech_420656.pcm')
('KaiSpeech_label_078903.txt', 'KaiSpeech_label_449461.txt', 'KaiSpeech_label_620819.txt', 'KaiSpeech_label_103058.txt', 'KaiSpeech_label_420656.txt')


In [11]:
from tqdm import trange

path = "E:/한국어 음성데이터/KaiSpeech/"
print('started...')
train_full = False
cnt = 0
length = len(audio_paths)
for idx in trange(length):
    audio = audio_paths[idx]
    label = label_paths[idx]
    if len(train_data_list['audio']) == TRAIN_NUM:
        train_full = True
    if train_full:
        test_data_list['audio'].append(audio)
        test_data_list['label'].append(label)
    else:
        rare_in = False
        f = open((path+audio).split('.')[0]+".txt")
        sentence = f.readline()
        f.close()
        for rare in rare_labels:
            if rare in sentence:
                rare_in = True
                break
        if rare_in:
            test_data_list['audio'].append(audio)
            test_data_list['label'].append(label)
        else:
            train_data_list['audio'].append(audio)
            train_data_list['label'].append(label)
print("Ended!!!")

started...


100%|███████████████████████████████████████████████████████████████████████| 622544/622544 [1:01:27<00:00, 168.85it/s]


Ended!!!


In [12]:
len(test_data_list['audio'])

148596

In [13]:
len(train_data_list['audio'])

498036

In [14]:
test_list = pd.DataFrame(test_data_list)
test_list.to_csv("test_list.csv", encoding='cp949', index=False)

In [15]:
train_list = pd.DataFrame(train_data_list)
train_list.to_csv("train_list.csv", encoding='cp949', index=False)